# Case Study 1 : Collecting Data from Twitter

Due Date: 2/1/2022, **BEFORE the beginning of class at 2:00pm EST**

## **NOTE: There are *always* last minute issues submitting the case studies.  DO NOT WAIT UNTIL THE LAST MINUTE!**

<img src="https://logos-download.com/wp-content/uploads/2016/02/Twitter_Logo_new-700x569.png">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    Abigail Albuquerque
    Aria Yan
    Isabel Herrero Estrada
    Megan Sin
    Sandra Phan
    

**Suggested Readings:** 
* Chapter 1 and Chapter 9 of the book "Mining the Social Web" can help a lot if you get stuck. 
* In fact, it is intentional that many of these questions can be answered directly from there (except for question 4)!
* The idea is to ease you into the case studies :-)

**Don't forget!**
* You will need to install the twitter library to access the Twitter API
 * pip install twitter
* NOTE: There is a package called "python-twitter" which will not work with this notebook!


** NOTE **
* **Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost**.

# Problem 1 (20 points): Sampling Twitter Data with the Search API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Search API to sample a collection of tweets about this topic. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [1]:
! pip install twitter

     |████████████████████████████████| 50 kB 4.0 MB/s 


In [ ]:
import twitter
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Prof. Paffenroth has a developer account for the class.  He will provide the Twitter access tokens for
    # each team
    # See https://developer.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 't9kxBUDYow0xyhlmrokijm8Sn'
    CONSUMER_SECRET = 'HRZc8ylwbeRIqnlZ44qFKwh5ZrR0Df2TK1WaKvpJzIJHGrAwWm'
    OAUTH_TOKEN = '571213367-MQTfjqVIK19sgUanvKtM6u2MiEJr65HSfSTgjRRf'
    OAUTH_TOKEN_SECRET = '1kQrgm98OgIYgNYxM7X6bOWUGagJgv8fMP2fRfS3dbSDb'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api


twitter_api = oauth_login()

In [73]:
import json
import time
from google.colab import drive

q = 'superbowl'
count = 100
search_results = twitter_api.search.tweets(q=q, count=count, lang='en')

statuses = search_results['statuses']


for _ in range(10): 
    print("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

status_texts = [ status['text'] 
                 for status in statuses ]

screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

words = [ w 
          for t in status_texts 
              for w in t.split() ]

drive.mount('/content/gdrive')
! ls gdrive/MyDrive/DS3010CaseStudy1Data

data= json.dumps(statuses, indent=1)

file = open('gdrive/MyDrive/DS3010CaseStudy1Data/superbowl.txt','w')
file.write(data)
file.close() 
#data saved to file, do not rerun - will change file list





Length of statuses 100
Length of statuses 200
Length of statuses 300
Length of statuses 400
Length of statuses 500
Length of statuses 599
Length of statuses 699
Length of statuses 799
Length of statuses 899
Length of statuses 999
Mounted at /content/gdrive
superbowl.txt


In [74]:
retweets = [status['retweet_count']
              for status in statuses]
print("search topic: " + q)
print("tweets: " + str(len(status_texts)))
print("word count: " + str(len(words)))
#print(json.dumps(status_texts[0:5], indent=1))
#print(json.dumps(screen_names[0:5], indent=1))
#print(json.dumps(hashtags[0:5], indent=1))
#print(json.dumps(words[0:5], indent=1))
#print(json.dumps(statuses[0], indent=4))

search topic: superbowl
tweets: 1099
word count: 19883


# Problem 2 (20 points): Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [75]:
#list of stopwords to filter out
#list taken from http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words

stopwords = ['a', 'about', 'above', 'across', 'after', 'afterwards']
stopwords += ['again', 'against', 'all', 'almost', 'alone', 'along']
stopwords += ['already', 'also', 'although', 'always', 'am', 'among']
stopwords += ['amongst', 'amoungst', 'amount', 'an', 'and', 'another']
stopwords += ['any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere']
stopwords += ['are', 'around', 'as', 'at', 'back', 'be', 'became']
stopwords += ['because', 'become', 'becomes', 'becoming', 'been']
stopwords += ['before', 'beforehand', 'behind', 'being', 'below']
stopwords += ['beside', 'besides', 'between', 'beyond', 'bill', 'both']
stopwords += ['bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant']
stopwords += ['co', 'computer', 'con', 'could', 'couldnt', 'cry', 'de']
stopwords += ['describe', 'detail', 'did', 'do', 'done', 'down', 'due']
stopwords += ['during', 'each', 'eg', 'eight', 'either', 'eleven', 'else']
stopwords += ['elsewhere', 'empty', 'enough', 'etc', 'even', 'ever']
stopwords += ['every', 'everyone', 'everything', 'everywhere', 'except']
stopwords += ['few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first']
stopwords += ['five', 'for', 'former', 'formerly', 'forty', 'found']
stopwords += ['four', 'from', 'front', 'full', 'further', 'get', 'give']
stopwords += ['go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her']
stopwords += ['here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers']
stopwords += ['herself', 'him', 'himself', 'his', 'how', 'however']
stopwords += ['hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed']
stopwords += ['interest', 'into', 'is', 'it', 'its', 'itself', 'keep']
stopwords += ['last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made']
stopwords += ['many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine']
stopwords += ['more', 'moreover', 'most', 'mostly', 'move', 'much']
stopwords += ['must', 'my', 'myself', 'name', 'namely', 'neither', 'never']
stopwords += ['nevertheless', 'next', 'nine', 'no', 'nobody', 'none']
stopwords += ['noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of']
stopwords += ['off', 'often', 'on','once', 'one', 'only', 'onto', 'or']
stopwords += ['other', 'others', 'otherwise', 'our', 'ours', 'ourselves']
stopwords += ['out', 'over', 'own', 'part', 'per', 'perhaps', 'please']
stopwords += ['put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed']
stopwords += ['seeming', 'seems', 'serious', 'several', 'she', 'should']
stopwords += ['show', 'side', 'since', 'sincere', 'six', 'sixty', 'so']
stopwords += ['some', 'somehow', 'someone', 'something', 'sometime']
stopwords += ['sometimes', 'somewhere', 'still', 'such', 'system', 'take']
stopwords += ['ten', 'than', 'that', 'the', 'their', 'them', 'themselves']
stopwords += ['then', 'thence', 'there', 'thereafter', 'thereby']
stopwords += ['therefore', 'therein', 'thereupon', 'these', 'they']
stopwords += ['thick', 'thin', 'third', 'this', 'those', 'though', 'three']
stopwords += ['three', 'through', 'throughout', 'thru', 'thus', 'to']
stopwords += ['together', 'too', 'top', 'toward', 'towards', 'twelve']
stopwords += ['twenty', 'two', 'un', 'under', 'until', 'up', 'upon']
stopwords += ['us', 'very', 'via', 'was', 'we', 'well', 'were', 'what']
stopwords += ['whatever', 'when', 'whence', 'whenever', 'where']
stopwords += ['whereafter', 'whereas', 'whereby', 'wherein', 'whereupon']
stopwords += ['wherever', 'whether', 'which', 'while', 'whither', 'who']
stopwords += ['whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with']
stopwords += ['within', 'without', 'would', 'yet', 'you', 'your']
stopwords += ['yours', 'yourself', 'yourselves']

In [76]:
def cleanWordList(words):
  return [w.lower() for w in words]

def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))

def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

def removeStopwords(wordlist, stopwords):
    return [w for w in wordlist if w not in stopwords]

wordlist = cleanWordList(words)

wordfreq = [words.count(w) for w in wordlist]
freqdict = wordListToFreqDict(wordlist)
sortedfreqdict = sortFreqDict(freqdict)
#print(sortedfreqdict[:20])

#print("remove stopwords")
filtwordlist = removeStopwords(wordlist, stopwords)
filwordfreq = [filtwordlist.count(w) for w in filtwordlist]
filtfreqdict = wordListToFreqDict(filtwordlist)
filtsortedfreqdict = sortFreqDict(filtfreqdict)
#print(filtsortedfreqdict[:30])

In [77]:
import pandas as pd

words = []
freq = []

for duo in filtsortedfreqdict[:30]:
  words.append(duo[1])
  freq.append(duo[0])

data = {'Frequency': freq,
        'Words': words}

df = pd.DataFrame(data)
print(df.to_string(index=False))

 Frequency                    Words
       625                       rt
       337                superbowl
       333               #superbowl
       121                      day
       108                  bengals
       106                      win
        89                     just
        82                 santonio
        79                       td
        69              spectacular
        69           roethlisberger
        69                   linked
        69                   holmes
        69                    grabs
        69                      ben
        69               @superbow…
        69             @nfllegends:
        69                    2009,
        64                 stafford
        60                      joe
        58                   burrow
        57                      lvi
        55                    years
        55                     game
        52                    super
        52                  matthew
        49                  

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [58]:


retweets = [status['retweet_count']
              for status in statuses]


rtdict = dict(list(zip(status_texts, retweets)))
def sortRTdict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux
sortedrtdict = sortRTdict(rtdict)
rts = []
tweets = []

for duo in sortedrtdict[:10]:
  tweets.append(duo[1])
  rts.append(duo[0])

printdata = {'Retweets': rts,
        'Tweet': tweets}

rtdf = pd.DataFrame(printdata)
print(rtdf.to_string(index=False))

 Retweets                                                                                                                                             Tweet
     6253                                         If you go from a 10 to a 5 because of light rain. Don't tell me who I can root for in the Superbowl 🤷🏼‍♂️
     6253       @BleacherReport @jalenramsey @brgridiron It had to be somebody. At least it was the GOAT. You can put that on your… https://t.co/wIiTDnQJ2v
     6253  #NFLPlayoffs Pass Update :\n4% #SuperBowl BEST BET Added\nvia @Greek_Gambler Card -\nhttps://t.co/l13VxT2j5x \n\n5% Big… https://t.co/4O6ZRylmMG
     3533                                  RT @kplightning: 7 years since we got the most watched superbowl performance of all time https://t.co/kUBAEO9I6d
     2605                                                RT @49ersSportsTalk: This is just beautiful… 🥺🥲\n#Bengals #SuperBowl #NFL  https://t.co/eau6wbIA0u
     2605      @TomBrady @Buccaneers Now that Bruce A. screwed u

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [ ]:

#wordlist = cleanWordList(words)

htfreq = [hashtags.count(ht) for ht in hashtags]
htdict = wordListToFreqDict(hashtags)
#print(htfreq)
sortedhtdict = sortFreqDict(htdict)
#print(sortedhtdict[:10])

htfreq = []
hashtags = []

for duo in sortedhtdict[:10]:
  hashtags.append(duo[1])
  htfreq.append(duo[0])

htdata = {'Hashtag Frequency': htfreq,
        'Hashtag': hashtags}

htdf = pd.DataFrame(htdata)
print(htdf.to_string(index=False))


In [ ]:
umfreq = [screen_names.count(um) for um in screen_names]
umdict = wordListToFreqDict(screen_names)
sortedumdict = sortFreqDict(umdict)
umfreq = []
usermentions = []

for duo in sortedumdict[:10]:
  usermentions.append(duo[1])
  umfreq.append(duo[0])

utdata = {'User Mention Frequency': umfreq,
          'Users': usermentions}

utdf = pd.DataFrame(utdata)
print(utdf.to_string(index=False))


* ------------------------

# Problem 3 (20 points): Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [ ]:
pip install tweepy[async]

In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

import tweepy

CONSUMER_KEY = 't9kxBUDYow0xyhlmrokijm8Sn'
CONSUMER_SECRET = 'HRZc8ylwbeRIqnlZ44qFKwh5ZrR0Df2TK1WaKvpJzIJHGrAwWm'
OAUTH_TOKEN = '571213367-MQTfjqVIK19sgUanvKtM6u2MiEJr65HSfSTgjRRf'
OAUTH_TOKEN_SECRET = '1kQrgm98OgIYgNYxM7X6bOWUGagJgv8fMP2fRfS3dbSDb'

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
  
# set access to user's access key and access secret 
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

# calling the api 
api = tweepy.API(auth)

# the screen_name of the targeted user
screen_name = "kendricklamar"

# get all friends
friends = tweepy.Cursor(api.friends, screen_name)

# get all followers
followers = tweepy.Cursor(api.followers, screen_name)


In [ ]:
# lists for follower screen names and ID numbers
follower_screen_names = []
follower_id = []

# adding the latest 20 followers of the user to lists
for follower in api.followers(screen_name):
  follower_screen_names.append(follower.screen_name)
  follower_id.append(follower.id)

# plot friends in table
import pandas as pd

data = {'Screen Name': follower_screen_names,
        'ID': follower_id}

df = pd.DataFrame(data)
print("20 Followers:")
print(df.to_string(index=False))

In [ ]:
# lists for friend screen names and ID numbers
friend_screen_names = []
friend_id = []

# adding the latest 20 friends of the user to lists
for friend in api.friends(screen_name):
  friend_screen_names.append(friend.screen_name)
  friend_id.append(friend.id)

# plot friends in table
import pandas as pd

data = {'Screen Name': friend_screen_names,
        'ID': friend_id}

df = pd.DataFrame(data)
print("20 Friends:")
print(df.to_string(index=False))

In [ ]:
if is_rate_limit_error_message(error_msg):
  raise RateLimitError(error_msg, resp)
else:
  raise TweepError(error_msg, resp, api_code=api_error_code)

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [ ]:
# lists for mutual screen names and ID numbers
mutual_screen_names = []
mutual_id = []

# loop through friends and add screen names and IDs for people who also follow back
for friend in api.friends(screen_name):
  status = api.show_friendship(api.get_user(screen_name).id, screen_name, friend.id, friend.screen_name)
  if(status[1].following == True):
    mutual_screen_names.append(status[1].screen_name)
    mutual_id.append(status[1].id)

# plot friends in table
import pandas as pd

data = {'Screen Name': mutual_screen_names,
        'ID': mutual_id}

df = pd.DataFrame(data)
print("Mutuals:")
print(df.to_string(index=False))

*------------------------

# Problem 4 (20 points): Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [36]:
import tweepy

CONSUMER_KEY = 't9kxBUDYow0xyhlmrokijm8Sn'
CONSUMER_SECRET = 'HRZc8ylwbeRIqnlZ44qFKwh5ZrR0Df2TK1WaKvpJzIJHGrAwWm'
OAUTH_TOKEN = '571213367-MQTfjqVIK19sgUanvKtM6u2MiEJr65HSfSTgjRRf'
OAUTH_TOKEN_SECRET = '1kQrgm98OgIYgNYxM7X6bOWUGagJgv8fMP2fRfS3dbSDb'

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
  
# set access to user's access key and access secret 
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

# calling the api 
API = tweepy.API(auth)

In [44]:
place = API.reverse_geocode(lat=42.1040, lon=-93.5003)
print(place)

[Place(_api=<tweepy.api.API object at 0x7f35ba85de10>, id='3cd4c18d3615bbc9', name='Iowa', full_name='Iowa, USA', country='United States', country_code='US', url='https://api.twitter.com/1.1/geo/id/3cd4c18d3615bbc9.json', place_type='admin', attributes={}, bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x7f35ba85de10>, type='Polygon', coordinates=[[[-96.6396669, 40.375437], [-96.6396669, 43.50102], [-90.140061, 43.50102], [-90.140061, 40.375437], [-96.6396669, 40.375437]]]), centroid=[-93.15059479194798, 41.938431], contained_within=[Place(_api=<tweepy.api.API object at 0x7f35ba85de10>, id='96683cc9126741d1', name='United States', full_name='United States', country='United States', country_code='US', url='https://api.twitter.com/1.1/geo/id/96683cc9126741d1.json', place_type='country', attributes={}, bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x7f35ba85de10>, type='Polygon', coordinates=[[[-179.231086, 13.182335], [-179.231086, 71.434357], [179.859685, 71.434357], 

In [2]:
import twitter
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Prof. Paffenroth has a developer account for the class.  He will provide the Twitter access tokens for
    # each team
    # See https://developer.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 't9kxBUDYow0xyhlmrokijm8Sn'
    CONSUMER_SECRET = 'HRZc8ylwbeRIqnlZ44qFKwh5ZrR0Df2TK1WaKvpJzIJHGrAwWm'
    OAUTH_TOKEN = '571213367-MQTfjqVIK19sgUanvKtM6u2MiEJr65HSfSTgjRRf'
    OAUTH_TOKEN_SECRET = '1kQrgm98OgIYgNYxM7X6bOWUGagJgv8fMP2fRfS3dbSDb'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api


twitter_api = oauth_login()

In [69]:
import json
import time

q = 'rams'
count = 100



search_results = twitter_api.search.tweets(q=q, count=count, lang='en')

statuses = search_results['statuses']

for _ in range(10): 
    print("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

status_texts = [ status['text'] 
                 for status in statuses ]

screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

words = [ w 
          for t in status_texts 
              for w in t.split() ]


#print(json.dumps(statuses[0], indent=4))

Length of statuses 100
Length of statuses 200
Length of statuses 300
Length of statuses 400
Length of statuses 500
Length of statuses 600
Length of statuses 700
Length of statuses 800
Length of statuses 900
Length of statuses 1000


In [70]:
wordlist = cleanWordList(words)

wordfreq = [words.count(w) for w in wordlist]
freqdict = wordListToFreqDict(wordlist)
sortedfreqdict = sortFreqDict(freqdict)
#print(sortedfreqdict[:20])

#print("remove stopwords")
filtwordlist = removeStopwords(wordlist, stopwords)
filwordfreq = [filtwordlist.count(w) for w in filtwordlist]
filtfreqdict = wordListToFreqDict(filtwordlist)
filtsortedfreqdict = sortFreqDict(filtfreqdict)
#print(filtsortedfreqdict[:30])

In [71]:
import pandas as pd

words = []
freq = []

for duo in filtsortedfreqdict[:30]:
  words.append(duo[1])
  freq.append(duo[0])

data = {'Frequency': freq,
        'Words': words}

df = pd.DataFrame(data)
print(df.to_string(index=False))

 Frequency                    Words
       664                     rams
       460                       rt
       189                    super
       177                      win
       176            @timthetatman
       162                    right
       132                     bowl
       116                    rams.
       115                  bengals
       104                     said
       102                    brady
        97                     mask
        95                     just
        91                     fans
        90                   newsom
        85                     lost
        81                     took
        81                    times
        81                     feel
        77                      100
        76                   worse.
        75  https://t.co/u5fu5fe1hi
        75        @realskipbayless:
        73                   picked
        62                    gavin
        55               california
        54             @clay

In [72]:
umfreq = [screen_names.count(um) for um in screen_names]
umdict = wordListToFreqDict(screen_names)
sortedumdict = sortFreqDict(umdict)
umfreq = []
usermentions = []

for duo in sortedumdict[:10]:
  usermentions.append(duo[1])
  umfreq.append(duo[0])

utdata = {'User Mention Frequency': umfreq,
          'Users': usermentions}

utdf = pd.DataFrame(utdata)
print(utdf.to_string(index=False))


 User Mention Frequency            Users
                    224     timthetatman
                     78  RealSkipBayless
                     54       ClayTravis
                     16     weddlesbeard
                     16          RamsNFL
                     15  WillLesterPhoto
                     14      ninernate49
                     13              PFF
                     13          Bengals
                     13     AroundTheNFL


*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10-15 minutes talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu) *and* the TA (dyou@wpi.edu).

#### We auto-process the submissions so make sure your subject line is *exactly*:

### DS3010 Case Study 1 Team ??

#### where ?? is your team number.
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 100 **


---------------------------------------------------------------------------
** Notebook results:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.





---------------------------------------------------------------------------
**Slides (for presentation): Story-telling**
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

